# Template for Creating Sacrament Meeting Programs

## Uses MarkupPy to generate the HTML

In [1]:
#
# Sugar Land 2nd Ward HTML Program Writer
#
# Pete Slater
# June 2022

import hymndict # Titles and links to hymns
import htmlpy # Local copy of source code
from htmlpy import oneliner as e
import artlinks # Links to artwork
import sys
import datetime as dt

# Customized functions to create the HTML for standard agenda items

In [2]:
'''
Define functions for formatting each of the items that can appear in a program.
Build the program in the main script by calling the functions.

'''
def officer(role, officer, page):
    page.p(e.b(role+":")+" "+officer,align="center")

def pagetitle(unit, meeting, meetdate,page):
    page.h1(unit, align="center")
    page.h2(meeting, align="center")
    page.h2(meetdate, align="center")

def speaker(name, page):
    page.h2("Speaker", align="center")
    page.p(name,align="center")
    
def testimonies(page):
    page.h2("Bearing of Testimonies", align="center")

def music(number, description, page):
    page.h2(description,align="center")
    [hymntitle, hymnurl] = hymndict.hymns[number]
    page.p(e.a("#" + str(number)+", "+hymntitle, href=hymnurl),align="center")
    
def specialmusic(performers,page,title=None,accompanist=None):
    page.h2("Special Musical Number",align="center")
    if title != None:
        page.p(e.i(title),align="center")
    page.p(performers,align="center")
    if accompanist != None:
        page.p("Acc. by " + accompanist, align="center")
        
def thought(text, author, page):
    page.hr()
    page.p(text, align="center")
    page.p("- "+author,align="center")
 
# Announcements are passed as a list of text strings
def announcements(textlist,page):
    page.hr()
    page.h1("Announcements", align="center")
    for txt in textlist:
        page.p(txt,align="center")

# Custom section for Preparedness Corner, etc.
def custom(title, txtlist, linklist, page):
    page.hr()
    page.h1(title, align="center")
    if txtlist != None:
        for txt in txtlist:
            page.p(txt,align="center")
    if linklist != None:
        for link in linklist:
            page.p(e.a(link[0], href=link[1], target="_blank", rel="noreferrer noopener"),align="center")
        
# Place some links, passed as a list containing text and url
def links(linklist, page):
    page.hr()
    page.h1("Links", align="center")
    for link in linklist:
        page.p(e.a(link[0], href=link[1], target="_blank", rel="noreferrer noopener"),align="center")
        
# Print credits at the bottom of the page
def credit(editor, page):
#'''Print credits and datestamp ''' 
    today = dt.date.today()
    page.hr()
    text = today.strftime("Edited %B %d, %Y by ") + editor
    page.small(text,align="center")        
    

# Format the agenda items that are always present
Customize for each week with date and type of meeting, i.e. sacrament, fast and testimony, ward conference

In [3]:
title = "Sugar Land Second Ward"
header = "Sacrament Meeting"
#header = "Fast and Testimony Meeting"
footer = ""
styles = ( 'layout.css', 'alt.css', 'images.css' )

page = htmlpy.page( )
page.init()
page.br( )
 
# Make sure it will look good on all devices
page.meta(name="viewport", content="width=device-width, initial-scale=1.0")

# Define the elements on the current week's programs here

pagetitle("Sugar Land 2nd Ward", "Sacrament Meeting", "December 11th, 2022", page)

## Artwork
Pick an artwork from the defined list of links in artlinks.py and enter the number here

In [4]:
# Place an artwork from the imported dictionary of links
#page.p(e.img(width=299, height=300*0.8, src=artlinks.art[2]), align="center")
page.p(e.img(style="max-width:50%;height:auto;", src=artlinks.art['Nativity1']), align="center")

## Officers

In [5]:
officer("Presiding","Bishop Joey Powell", page)
#officer("Conducting","Bishop Joey Powell", page)
officer("Conducting","Brother Jared Draney", page)
#officer("Conducting","Brother Jared Draney", page)

## Opening and sacrament hymns

In [6]:
music(212,"Opening Hymn",page)
music(188,"Sacrament Hymn", page) 

## Variable part of the program

### If it's fast and testimony meeting, uncomment this section and comment out the sacrament meeting items

In [7]:
# Uncomment this line for fast and testimony meeting
#testimonies(page)

### For a sacrament meeting or ward conference, comment out the testimonies and use these sections as needed

In [8]:
# Use lines like these to form the speaking part of the program

speaker("Veniece Griffin", page)
#speaker("Wendy Markwalter", page)

# Special code for the Primary Program
#txtlist = (
#    'Hymns related to the Old Testament',
#    'Program under the direction of Kaylee Robinson',
#    'Music under the direction of Veniece Griffin and Dawnette Moore',
#    'Congregation to join the children in singing when directed.'
#    )
#linklist = None

#custom("Primary Program", txtlist,linklist, page)

#music(93,"Intermediate Hymn",page)
specialmusic("Schuyler and Kaylee Robinson",page, title="What Child Is This?",accompanist = "Dallin Arnold")

speaker("Jared Draney", page)
#speaker("Ben Powell", page)

### Closing hymn

In [9]:
music(209,"Closing Hymn",page)

## End matter - thought, announcements, links

In [10]:
# Spiritual thought
quote = ('Being grateful in times of distress does not mean we are pleased with our circumstances. It does mean that through the eyes of faith we look beyond our present-day challenges. This is not a gratitude of the lips but of the soul. It is a gratitude that heals the heart and expands the mind.')
author = 'Dieter F. Uchtdorf'
thought(quote, author, page)

# Make a list of announcements, then post to the page
txtlist = (
            "Sunday, December 11, is the last day for Tithing Declaration. Sign up on the bishop's door",
            "December 11 is the last day to donate to the Angel Tree - Take an ornament from the Angel Tree and support Ft. Bend Rainbow Room",
            "Stake Christmas Concert, Sunday, December 11, 7:00pm",
            "Send announcements to bvl2clerk@gmail.com"
           )
announcements(txtlist, page)

### Preparedness Corner - under development

In [11]:
#txtlist = (
#    'Make sure you have water: one gallon per person per day. A family of four would need 12 gallons for 3 days.',
#    ' '
#    )
#linklist = {
#    ("Learn more online","https://www.churchofjesuschrist.org/study/manual/gospel-topics/food-storage?lang=eng")
#}
#custom("Preparedness Corner", txtlist,linklist, page)

In [12]:
# Make a list of links, then post
linklist = (
        ("Donate at the Houston Giving Machine","https://www.givingmachineshouston.org/"),
        ("Read the First Presidency''s Christmas Message","https://newsroom.churchofjesuschrist.org/article/first-presidencys-2022-christmas-message"),
        ("Sign up to feed the missionaries","https://www.signupgenius.com/go/10c0e4baca82ea7fb6-dinner2"),
        ("Sign up to prepare a meal with Fort Bend Family Promise","https://www.signupgenius.com/go/10c0e4baca82ea7fb6-helping4"),
        ("Help at secondmile.org","https://secondmile.org"), 
        )
links(linklist, page)
credit("Pete Slater and Andy Chen", page)

# Write out the file to Index.html, which must be posted online

In [13]:
print (page)# -*- coding: utf-8 -*

original_stdout = sys.stdout
with open('Index.html', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(page)
    sys.stdout = original_stdout # Reset the standard output to its original value
"""
End of script
"""

<!DOCTYPE HTML PUBLIC '-//W3C//DTD HTML 4.01 Transitional//EN'>
<html lang="en">
<head>
</head>
<body>
<br />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<h1 align="center">Sugar Land 2nd Ward</h1>
<h2 align="center">Sacrament Meeting</h2>
<h2 align="center">December 11th, 2022</h2>
<p align="center"><img style="max-width:50%;height:auto;" src="https://news-uk.churchofjesuschrist.org/media/960x540/nativity-2013.jpg" /></p>
<p align="center"><b>Presiding:</b> Bishop Joey Powell</p>
<p align="center"><b>Conducting:</b> Brother Jared Draney</p>
<h2 align="center">Opening Hymn</h2>
<p align="center"><a href="https://www.churchofjesuschrist.org/music/library/hymns/far-far-away-on-judeas-plains?lang=eng&amp;#39">#212, Far, Far Away on Judea's Plains</a></p>
<h2 align="center">Sacrament Hymn</h2>
<p align="center"><a href="https://www.churchofjesuschrist.org/music/library/hymns/thy-will-o-lord-be-done?lang=eng&amp;#39">#188, Thy Will, O Lord, Be Done</a></p>
<h2 a

'\nEnd of script\n'